In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

data = pd.read_csv("DATA/Minmax_Scaler_result.csv",encoding="cp949")
data.head()

,CUSTNO,GNO,CBSCORE,CBSCOREGRD,CREDITOTAMT,YSALEAMT,ESTMM,ASSETAMT,IMSAAMT,IMJUAMT,...,환율,GDP,소상공인체감지수,실업률,물가지수,국고채,금리,유가등락률,소비자심리지수,ONEHOT
0,475821,l180202101898,0.949290,0.000000,0.002338,0.223956,0.058085,0.000768,0.004153,0.000000,...,0.436398,0.989382,0.320660,0.346154,0.928674,0.144172,0.000000,0.511688,0.956072,3
1,74417,l230201700120,0.897566,0.166667,0.002221,0.129121,0.014129,0.001920,0.000000,0.015489,...,0.518949,0.691828,0.355093,0.423077,0.707266,0.257669,0.272727,0.423701,0.545220,8
2,387787,l110201603233,0.909736,0.000000,0.000000,0.083846,0.290424,0.000768,0.004153,0.000000,...,0.587992,0.661863,0.624821,0.346154,0.634454,0.239264,0.272727,0.289610,0.578811,7
3,395418,l230201700254,0.884381,0.166667,0.001637,0.079560,0.017268,0.000768,0.004153,0.000000,...,0.516323,0.691828,0.676471,0.307692,0.703799,0.269939,0.272727,0.332792,0.604651,3
4,190372,l200201601418,0.726166,0.833333,0.000818,0.036813,0.262166,0.006143,0.000831,0.000000,...,0.504690,0.643228,0.489957,0.307692,0.592352,0.119632,0.272727,0.190909,0.715762,7


In [2]:
features = data[['CBSCORE', 'CREDITOTAMT', 'YSALEAMT',
       'ESTMM',  'IMSAAMT', 'IMJUAMT', 'BUSAAMT', 'BUJUAMT',
       'BU1TOTAMT', 'GAMT', 'LABORCNT', 'KOSPI', '환율', 'GDP', 
       '소상공인체감지수', '실업률', '물가지수', '국고채', '금리', '유가등락률',
       '소비자심리지수','ONEHOT']]
act = data['ACTCD']

In [3]:
from sklearn.model_selection import train_test_split

train_x, test_x,train_y, test_y = train_test_split(features, act,
stratify=act,train_size=0.7,test_size=0.3,random_state=1)

print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(27381, 22) (11736, 22) (27381,) (11736,)


In [4]:
import statsmodels.api as sm
from imblearn.over_sampling import BorderlineSMOTE

sm = BorderlineSMOTE(random_state=42)
over_train_x, over_train_y = sm.fit_resample(train_x,train_y)

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier

log_clf = LogisticRegression(random_state = 42, solver = 'lbfgs', max_iter = 3000)
rnd_clf = RandomForestClassifier(n_estimators=200,max_depth=50,max_features=1)
knn_clf = KNeighborsClassifier(n_neighbors= 2,algorithm='kd_tree',weights='uniform')
rnd_clf_1 = RandomForestClassifier(n_estimators=200,max_depth=50,max_features=2)

voting_clf = VotingClassifier(
    estimators = [('lr', log_clf), ('rf', rnd_clf), ('knn', knn_clf),('rf2', knn_clf)],
    voting = 'hard'
)

voting_clf.fit(over_train_x,over_train_y)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(max_iter=3000,
                                                 random_state=42)),
                             ('rf',
                              RandomForestClassifier(max_depth=50,
                                                     max_features=1,
                                                     n_estimators=200)),
                             ('knn',
                              KNeighborsClassifier(algorithm='kd_tree',
                                                   n_neighbors=2)),
                             ('rf2',
                              KNeighborsClassifier(algorithm='kd_tree',
                                                   n_neighbors=2))])

In [7]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

for clf in (log_clf, rnd_clf, knn_clf, voting_clf):
    clf.fit(over_train_x, over_train_y)
    y_pred = clf.predict(test_x)
    cfmat = confusion_matrix(test_y,y_pred)
    print(clf)
    print(cfmat)
    print(classification_report(test_y, y_pred, target_names=['정상보증', '사고보증']))

LogisticRegression(max_iter=3000, random_state=42)
[[7443 3223]
 [ 360  710]]
              precision    recall  f1-score   support

        정상보증       0.95      0.70      0.81     10666
        사고보증       0.18      0.66      0.28      1070

    accuracy                           0.69     11736
   macro avg       0.57      0.68      0.54     11736
weighted avg       0.88      0.69      0.76     11736

RandomForestClassifier(max_depth=50, max_features=1, n_estimators=200)
[[10362   304]
 [  750   320]]
              precision    recall  f1-score   support

        정상보증       0.93      0.97      0.95     10666
        사고보증       0.51      0.30      0.38      1070

    accuracy                           0.91     11736
   macro avg       0.72      0.64      0.66     11736
weighted avg       0.89      0.91      0.90     11736

KNeighborsClassifier(algorithm='kd_tree', n_neighbors=2)
[[9654 1012]
 [ 698  372]]
              precision    recall  f1-score   support

        정상보증       0.93    